## HMM模型


隐马尔可夫模型是关于时序的概率模型，描述由一个隐藏的马尔可夫链随机生成不可观测的状态随机序列，再由各个状态生成一个观测随机序列的过程。

隐藏的马尔可夫链链随机生成的状态序列，称为状态序列；每个状态生成一个观测，而由此产生的观测随机序列，称为观测序列。

模型的参数包括状态转移矩阵A，观测概率矩阵B和初始状态概率向量Π。

从定义可知，HMM作了两个基本假设
+ 齐次马尔可夫性假设，即每时刻的状态只与前一时刻有关
+ 观测独立性假设，即任意时刻的观测只依赖于该时刻的状态

In [1]:
import numpy as np

In [14]:
A = np.array(([0.5,0.2,0.3],[0.3,0.5,0.2],[0.2,0.3,0.5]))
B = np.array(([0.5,0.5],[0.4,0.6],[0.7,0.3]))
pi = np.array([0.2,0.4,0.4])

### 前向算法

In [35]:
# 前向传播算法
def forward(A,B,pi,O):
    '''
    A,B,pi是HMM的参数
    O是观测序列
    '''
    T = len(O)
    N, M = B.shape # N,M分别是状态数和观测结果数
    alpha = np.zeros((N,T)) # 用于记录前向概率
    a = B[:,O[0]]*pi
    alpha[:,0] = a
    for i in range(1,T):
        a = np.dot(a.reshape([1,-1]),A)*B[:,O[i]]
        alpha[:,i] = a
    print(alpha)
    return np.sum(a)

In [36]:
O = [0,1,0,1]
forward(A,B,pi,O)

[[0.1        0.077      0.04187    0.0210779 ]
 [0.16       0.1104     0.035512   0.02518848]
 [0.28       0.0606     0.052836   0.01382442]]


0.06009079999999999

### 后向算法

In [59]:
def backward(A,B,pi,O):
    '''
    A,B,pi是HMM的参数
    O是观测序列
    '''
    T = len(O)
    N, M = B.shape # N,M分别是状态数和观测结果数
    beta = np.zeros((N,T))
    b = np.ones(N)
    beta[:,-1] = b
    for i in range(T-2,-1,-1):
        b = np.dot(A,b*B[:,O[i+1]])
        beta[:,i] = b
    print(beta)
    return np.dot(pi,b*B[:,O[0]])

In [60]:
backward(A,B,pi,O)

[[0.112462 0.2461   0.46     1.      ]
 [0.121737 0.2312   0.51     1.      ]
 [0.104881 0.2577   0.43     1.      ]]


0.06009079999999999

### 维特比算法

In [73]:
def vit(A,B,pi,O):
    '''
    A,B,pi是HMM的参数
    O是观测序列
    '''
    T = len(O)
    N, M = B.shape # N,M分别是状态数和观测结果数
    V = np.zeros((N,T))
    v = B[:,O[0]]*pi
    for i in range(1,T):
        print(np.argmax(v.reshape([-1,1])*A,axis=0))
        v = np.max(v.reshape([-1,1])*A,axis=0)*B[:,O[i]]
        V[:,i] = v
    return V
    

In [74]:
vit(A,B,pi,O)

[2 2 2]
[1 1 2]
[0 1 2]


array([[0.      , 0.028   , 0.00756 , 0.00189 ],
       [0.      , 0.0504  , 0.01008 , 0.003024],
       [0.      , 0.042   , 0.0147  , 0.002205]])